# Tarea 2 - Procesamiento distribuido y redes neuronales profundas
**Integrantes:** 

Camila Goméz Nazal -
Ignacio Zurita Tapia

In [ ]:
import torch
from torchvision.datasets import DatasetFolder
from torchvision.transforms import Compose, RandomApply, RandomHorizontalFlip, Resize, RandomRotation, Lambda
from torch.utils.data import DataLoader
from IPython.display import HTML
from torchvision import get_image_backend
seed = 81818 
torch.random.manual_seed(seed)

## Carga y transformación de datos


#### 1. Dataset Folder

In [3]:
def loader(path):
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

In [4]:
scale = Lambda(lambda x: torch.div(x, torch.max(x)))
brightness = Lambda(lambda x: torch.mult(x, torch.FloatTensor(x.shape[0], x.shape[1], x.shape[2]).uniform_(1.2, 1.5)))

In [30]:
root = 'ChestXRay2017/chest_xray'
extensions = ('.jpeg')
transform = Compose([Resize([224,224]),
                     scale,
                     RandomHorizontalFlip(0.5),
                     RandomRotation([-20,20]),
                     brightness])

train = DatasetFolder(root + '/train', loader, extensions, transform)
test = DatasetFolder(root + '/test', loader, extensions, transform)

#### 2. Cantidad de muestras en cada clase en train y test:



In [29]:
HTML(filename='distribucion_datos.html')

,Normal,Neumonía
Train,1349,3884
Test,234,390


#### 3.

#### 4.

## Redes neuronales profundas

## Interpretabilidad